In [ ]:
from google.cloud import bigquery
import pandas as pd
from annotationHelperLib import *
from reportMarkingFunctions import *
from IPython.display import clear_output
import json

In [ ]:
# Replace with the name you want used in publications of this data set
name = "Jenna Schabdach"

# Projects: SLIP, SLIP PreK, and SLIP Adolescents
project = "SLIP Adolescents"

# Replace with the path to your .json file containing a dictionary of all the phrases you'd like to be highlighted
phrasesToHighlightFn = "phrases_to_highlight.json"

# Load the dictionary of phrases to highlight in certain colors 
with open(phrasesToHighlightFn, 'r', encoding='utf-8') as f:
    toHighlight = json.load(f)
    
getGraderStatusReport(name)


In [ ]:
isOnMainTable = welcomeUser(name)

In [ ]:
batchSize = 10

for i in range(batchSize):
    clear_output()
    print(i)
    if isOnMainTable:
        markOneReportSQL(name, project, toHighlight)
    else:
        markSelfEvalReportSQL(name, toHighlight)


In [ ]:
# If/when you run out of reports, you can uncomment the following line of code
#   and run this cell to get 50 more reports from the database. Make sure your
#   values for `project` and `queryFn` match the project you're working on.
# Projects: SLIP, SLIP PreK, and SLIP Adolescents
# Queries: ./queries/slip_base.txt, ./queries/start2018_ages2-5years.txt, and ./queries/start2023_ages12-20years.txt
project = "SLIP Adolescents"
queryFn = "./queries/start2022_ages12-20years.txt"
getMoreReportsToGrade(name, project, queryFn, 10)